# ML Project: eCommerce Inventory Prescription

## Sources:

1. Getting cass index: https://fred.stlouisfed.org/series/FRGSHPUSM649NCIS
2. Insight into potential predictive features: https://dspace.mit.edu/bitstream/handle/1721.1/126484/scm2020-huang-a-predictive-model-for-transpacific-eastbound-ocean-freight-pricing-capstone.pdf?sequence=1&isAllowed=y
3. crude oil prices: https://www.indexmundi.com/commodities/?commodity=crude-oil&months=360
4. OLD: exchange rate: https://fred.stlouisfed.org/series/EXCHUS
5. US CPI: https://fred.stlouisfed.org/series/CPIAUCSL
6. US PMI: https://data.nasdaq.com/data/ISM/MAN_PMI-pmi-composite-index
7. OLD: China PMI:


## Data / Methodolgy

We need to predict the average US freight cost for a three month horizon. For the purposes of our problem, all items travel the same distance from the manufacturer to the warehouse. Since we are assuming that we are shipping less-than-truckload (LTL) shipments, cost is determined solely by weight of shipped items. We are using the CASS index to determine historical freight prices in the US. (insert explanation of CASS index). We can then estimate transportation costs accordingly: find average shipping cost per pound for the last month of our training set ($\text{\$/lb}_{N}$), associated with $\text{CASS Index}_{N}$, and back-compute all prior shipping costs per pound using the relationship between $\text{CASS Index}_{N}$ and the CASS index of the prior months.

To "predict" shipping rate, we will use the following features:
- oil price
- exchange rate (CH/US)   (might not need this one)
- US CPI
- US PMI
- Year
- Month

We got the above data from (sources). We found our other data (demand and individual item costs) from... (Semi)

Simple linear regression of the non-aggregated data shows that there is explanability in these features (in-sample R^2 of 0.5). 

Since we have a year of data, we can run the optimization 4 times, since our horizon is 1 quarter. Thus, we need 4 test periods, with each period correponding to 13 weeks of data. 

We'll break down the demand into 4 chunks of 13 weeks each (13 x 4 = 52). We can accordingly group the CASS data (response and features) into 3 month periods (quarters, thus 4 for each year), with the features being:
- Year
- Quarter
- Following features for the beginning of the quarter (taken from first month of quarter)
    - oil price
    - exchange rate (CH/US)   (might not need this one)
    - US CPI
    - US PMI 
    
This will let us find the neighbors of the 4 test quarters, since at the beginning of these quarters we will know the values of all the features. 

Simple linear regression of aggregated (quarterized) data leads to train R-squared of 0.5511. 

Essentially our analysis is by quarter, so we'll have projected profit (using prescription), baseline profit (simply using predicted avg CASS for quater), and ideal profit (oracle approach) for all four quarters.


Thus, work to do right now is:
- break down predicted demand into 4 matrices (19 x 13) each (we'll have to throw a week out, or include it in the last quarter or something)
- rewrite model to incorporate shipping costs per pound estimate and to account for change in horizon (change lead time to two weeks) 
- find simple prediction and oracle profits

In [175]:
using JuMP
using Gurobi
using CSV
using DataFrames
using Random, Statistics
using NearestNeighbors, Dates
ENV["COLUMNS"]=120;
gurobi_env = Gurobi.Env();

Academic license - for non-commercial use only - expires 2022-08-19


#### decision vars ####
x[i,t] --> inventory order from Supplier of product i at time t

j[i,t] --> how much we are selling of product i in time t


#### technically decision vars but not really decisions ####
s[i,t] --> inventory available for sale (at AMZ warehouse) for product i at time t

m[t] --> capital (money) available to purchase inventory at time t

#### params ####
d[i,t] --> demand for product i at time t

r[i] --> sales price for product i

c[i] --> manufacturing cost for product i

tr[i] --> transportation cost for product i


we'll incorporate volume later - for initial, lets assume fixed unit costs

v[i] --> volume (size) of product i

fba --> AMZ storage fee: $19/cbm per month



revenue = 0.70 * sum( d[i,t]*r[i] for i=1:items_tot, t=1:time_tot ) --> amazon takes 30% cut

cost = sum( x[i,t]*(c[i]+tr[i]) for i=1:items_tot, t=1:time_tot ) + sum( fba*s[i,t] for i=1:items_tot, t=1:time_tot:4 )

    --> manu + transport cost of orders + monthly inventory fee for whatever inventory we have


Transportation cost is something that we want to predict -- 

w[t] = total weight of items being shipped in week t
    
$\hat{S}$[m] = uncertain shipping cost in month m - found using CASS index
    
 
 Thus, total trans cost for week t = 
 
 w[t] * S[m(t)]

## RUNNING MODEL

In [176]:
#### load data ####

# train/tests for finding KNN
trainQ1 = CSV.read("data/data_final_train_Q1.csv", DataFrame)[:,2:end]
testQ1 = CSV.read("data/data_final_test_Q1.csv", DataFrame)
trainQ2 = CSV.read("data/data_final_train_Q2.csv", DataFrame)[:,2:end]
testQ2 = CSV.read("data/data_final_test_Q2.csv", DataFrame)
trainQ3 = CSV.read("data/data_final_train_Q3.csv", DataFrame)[:,2:end]
testQ3 = CSV.read("data/data_final_test_Q3.csv", DataFrame)
trainQ4 = CSV.read("data/data_final_train_Q4.csv", DataFrame)[:,2:end]
testQ4 = CSV.read("data/data_final_test_Q4.csv", DataFrame)


# demands
demand_Q1 = CSV.read("data/salesByWeek_Q1.csv", DataFrame)
demand_Q2 = CSV.read("data/salesByWeek_Q2.csv", DataFrame)
demand_Q3 = CSV.read("data/salesByWeek_Q3.csv", DataFrame)
demand_Q4 = CSV.read("data/salesByWeek_Q4.csv", DataFrame)
d1 = Matrix(demand_Q1)[:,3:end]
d2 = Matrix(demand_Q2)[:,3:end]
d3 = Matrix(demand_Q3)[:,3:end]
d4 = Matrix(demand_Q4)[:,3:end]
Q1_startingInv = sum(d1[:,1:2], dims=2);
Q2_startingInv = sum(d2[:,1:2], dims=2);
Q3_startingInv = sum(d3[:,1:2], dims=2);
Q4_startingInv = sum(d4[:,1:2], dims=2);


# fixed item prices and fees
prices = CSV.read("data/prices.csv", DataFrame)
itemInfo = CSV.read("data/costEstimates.csv", DataFrame)

# set lead time
leadtime=2

# get constant vectors from data
r = itemInfo[:,2]
c = itemInfo[:,3]
f = itemInfo[:,5]
FBA = itemInfo[:,7]
w = itemInfo[:,10];

In [180]:
## WITHOUT PRESCRIPTION

# model function
function runModel(startingCap, startingInv, nextQstartingInv, D, U)

    P = size(D)[1]
    T = size(D)[2]

    modelNoPres = Model(with_optimizer(Gurobi.Optimizer,TimeLimit=60, gurobi_env));
    set_optimizer_attribute(modelNoPres, "OutputFlag", 0)

    @variable(modelNoPres, x[i=1:P, t=1:T] >= 0) # main decision
    @variable(modelNoPres, j[i=1:P, t=1:T] >= 0) 
    @variable(modelNoPres, s[i=1:P, t=1:T] >= 0) 
    @variable(modelNoPres, m[t=1:T])
    @variable(modelNoPres, lambda[t=1:T] >= 0)

    # profit (capital) in every week = last week's + 2 weeks ago's sales - this weeks cost
    @constraint(modelNoPres, [t=1:T-2], m[t+2] == m[t+1] + sum(j[i,t]*(r[i] - f[i]) for i=1:P) - lambda[t+1])
    @constraint(modelNoPres, m[1] == startingCap)
    @constraint(modelNoPres, m[2] == m[1] - lambda[1])
    # cost in week is equal to purchasing cost + trans cost + storage cost
    @constraint(modelNoPres, [t=1:T], lambda[t] == sum(x[i,t]*c[i] for i=1:P) + U*sum(x[i,t]*w[i] for i=1:P) + sum(s[i,t]*FBA[i] for i=1:P))
    # sales capped by storage and demand
    @constraint(modelNoPres, [i=1:P,t=1:T], j[i,t] <= s[i,t])
    @constraint(modelNoPres, [i=1:P,t=1:T], j[i,t] <= D[i,t])
    # cannot spend more than we have in capital at beginning of week
    #@constraint(modelNoPres, [t=1:T], lambda[t] <= m[t])
    # inventory in week is equal to prev week inv - what we sold last week + what we ordered two weeks ago
    @constraint(modelNoPres, [i=1:P,t=leadtime:T-1], s[i,t+1] == s[i,t] - j[i,t] + x[i,t-1])
    @constraint(modelNoPres, [i=1:P], s[i,2] == s[i,1] - j[i,1])
    @constraint(modelNoPres, [i=1:P], s[i,1] == startingInv[i])
    # ending inventory (T=14) must be enough to start next quarter (whatever is left after last week
    # + what we ordered in the second to last week)
    @constraint(modelNoPres, [i=1:P], s[i,T] - j[i,T] + x[i,T-1] >= nextQstartingInv[i])
    
    @objective(modelNoPres, Max, m[T] + sum(j[i,T-1]*(r[i] - f[i]) for i=1:P) - lambda[T])
    #@objective(modelNoPres, Max, m[T])
    
    optimize!(modelNoPres)

    #termination_status(model)
    return objective_value(modelNoPres), value.(x)
    
end

runModel (generic function with 1 method)

# TEST SECTION

In [164]:
# testing it without constraints on capital
# ORACLE
Q1_startingInv = zeros(19)
U1_true = testQ1[1,7]
testProf, test_x, test_j, test_s, test_m, test_lambda = runModel(0, Q1_startingInv, Q2_startingInv, d1, U1_true);

In [165]:
testProf

1.5056136948115781e6

In [166]:
test_m

13-element Vector{Float64}:
       0.0
 -158386.32182924214
 -334830.748491804
 -520660.0759609932
 -439952.0333377179
 -322265.9298210073
 -181579.23283077485
   38823.180192610074
  227058.67258879324
  421154.2937119269
  619434.7219212883
       1.1367389899212883e6
  996037.3648115781

In [167]:
test_j

19×13 Matrix{Float64}:
 0.0  0.0   40.0   65.0   50.0   58.0   61.0   58.0   70.0   86.0   67.0   75.0  0.0
 0.0  0.0   68.0   61.0   78.0  118.0  109.0   99.0   97.0  109.0  111.0  108.0  0.0
 0.0  0.0   97.0   97.0   91.0  117.0  116.0  140.0  120.0  161.0  138.0  127.0  0.0
 0.0  0.0   73.0   69.0   65.0  102.0  117.0  117.0   86.0   97.0  123.0  125.0  0.0
 0.0  0.0  335.0  360.0  327.0  407.0  437.0  426.0  487.0  498.0  516.0  522.0  0.0
 0.0  0.0  364.0  376.0  398.0  427.0  557.0  525.0  457.0  527.0  564.0  577.0  0.0
 0.0  0.0  184.0  166.0  217.0  246.0  254.0  262.0  260.0  286.0  331.0  289.0  0.0
 0.0  0.0  161.0  148.0  158.0  212.0  215.0  196.0  225.0  279.0  262.0  289.0  0.0
 0.0  0.0   60.0   58.0   57.0   68.0   90.0   76.0   88.0   81.0   84.0   75.0  0.0
 0.0  0.0   71.0   72.0   84.0   94.0   92.0   94.0  104.0  108.0   96.0  102.0  0.0
 0.0  0.0   73.0  105.0  104.0  131.0   96.0  102.0  111.0  116.0  116.0  114.0  0.0
 0.0  0.0   11.0    8.0    5.0   11.0   11

In [168]:
test_x

19×13 Matrix{Float64}:
  40.0   65.0   50.0   58.0   61.0   58.0   70.0   86.0   67.0   75.0  0.0   185.0  0.0
  68.0   61.0   78.0  118.0  109.0   99.0   97.0  109.0  111.0  108.0  0.0   249.0  0.0
  97.0   97.0   91.0  117.0  116.0  140.0  120.0  161.0  138.0  127.0  0.0   330.0  0.0
  73.0   69.0   65.0  102.0  117.0  117.0   86.0   97.0  123.0  125.0  0.0   318.0  0.0
 335.0  360.0  327.0  407.0  437.0  426.0  487.0  498.0  516.0  522.0  0.0  1316.0  0.0
 364.0  376.0  398.0  427.0  557.0  525.0  457.0  527.0  564.0  577.0  0.0  1307.0  0.0
 184.0  166.0  217.0  246.0  254.0  262.0  260.0  286.0  331.0  289.0  0.0   702.0  0.0
 161.0  148.0  158.0  212.0  215.0  196.0  225.0  279.0  262.0  289.0  0.0   607.0  0.0
  60.0   58.0   57.0   68.0   90.0   76.0   88.0   81.0   84.0   75.0  0.0   200.0  0.0
  71.0   72.0   84.0   94.0   92.0   94.0  104.0  108.0   96.0  102.0  0.0   266.0  0.0
  73.0  105.0  104.0  131.0   96.0  102.0  111.0  116.0  116.0  114.0  0.0   320.0  0.0
  11.0   

In [169]:
test_lambda

13-element Vector{Float64}:
 158386.32182924214
 176444.42666256186
 185829.32746918921
 231186.9673767247
 230350.46648328935
 224235.98300976757
 233580.06697661508
 264257.69760381687
 245878.37887686642
 260031.63179063858
   1414.7320000000002
 623189.3051097101
      0.0

In [170]:
test_s

19×13 Matrix{Float64}:
 0.0  0.0   40.0   65.0   50.0   58.0   61.0   58.0   70.0   86.0   67.0   75.0  0.0
 0.0  0.0   68.0   61.0   78.0  118.0  109.0   99.0   97.0  109.0  111.0  108.0  0.0
 0.0  0.0   97.0   97.0   91.0  117.0  116.0  140.0  120.0  161.0  138.0  127.0  0.0
 0.0  0.0   73.0   69.0   65.0  102.0  117.0  117.0   86.0   97.0  123.0  125.0  0.0
 0.0  0.0  335.0  360.0  327.0  407.0  437.0  426.0  487.0  498.0  516.0  522.0  0.0
 0.0  0.0  364.0  376.0  398.0  427.0  557.0  525.0  457.0  527.0  564.0  577.0  0.0
 0.0  0.0  184.0  166.0  217.0  246.0  254.0  262.0  260.0  286.0  331.0  289.0  0.0
 0.0  0.0  161.0  148.0  158.0  212.0  215.0  196.0  225.0  279.0  262.0  289.0  0.0
 0.0  0.0   60.0   58.0   57.0   68.0   90.0   76.0   88.0   81.0   84.0   75.0  0.0
 0.0  0.0   71.0   72.0   84.0   94.0   92.0   94.0  104.0  108.0   96.0  102.0  0.0
 0.0  0.0   73.0  105.0  104.0  131.0   96.0  102.0  111.0  116.0  116.0  114.0  0.0
 0.0  0.0   11.0    8.0    5.0   11.0   11

In [171]:
nextperstartinginv = test_s[:,13] .- test_j[:,13] .+ test_x[:,13-1]

19-element Vector{Float64}:
  185.0
  249.0
  330.0
  318.0
 1316.0
 1307.0
  702.0
  607.0
  200.0
  266.0
  320.0
   33.0
   36.0
 1083.0
  186.0
  175.0
 1072.0
  105.0
  982.0

In [172]:
Q2_startingInv

19×1 Matrix{Int64}:
  185
  249
  330
  318
 1316
 1307
  702
  607
  200
  266
  320
   33
   36
 1083
  186
  175
 1072
  105
  982

# END TEST SECTION

In [213]:
## WITH PRESCRIPTION

# model function
function runModelPres(startingCap, startingInv, nextQstartingInv, D, U)

    model = Model(with_optimizer(Gurobi.Optimizer,TimeLimit=60, gurobi_env));
    set_optimizer_attribute(model, "OutputFlag", 0)
    
    if isa(U, Vector) == true
        K = size(U)[1]
    else
        K = 1
    end
    
    neighborWeight = zeros(K)
    neighborWeight .= 1/K
    
    P = size(D)[1]
    T = size(D)[2]

    @variable(model, x[i=1:P, t=1:T] >= 0) # Main decision
    @variable(model, j[i=1:P, t=1:T, k=1:K] >= 0) 
    @variable(model, s[i=1:P, t=1:T, k=1:K] >= 0) 
    @variable(model, m[t=1:T, k=1:K]) # can be in debt
    @variable(model, lambda[t=1:T, k=1:K] >= 0)

    # profit (capital) in every week = last week's + 2 weeks ago's sales - this weeks cost
    @constraint(model, [t=1:T-2, k=1:K], m[t+2,k] == m[t+1,k] + sum(j[i,t,k]*(r[i] - f[i]) for i=1:P) - lambda[t+1,k])
    @constraint(model, [k=1:K], m[1,k] == startingCap)
    @constraint(model, [k=1:K], m[2,k] == m[1,k] - lambda[1,k])
    # cost in week is equal to purchasing cost + trans cost + storage cost
    @constraint(model, [t=1:T,k=1:K], lambda[t,k] == sum(x[i,t]*c[i] for i=1:P) + U[k]*sum(x[i,t]*w[i] for i=1:P) + sum(s[i,t,k]*FBA[i] for i=1:P))
    # sales capped by storage and demand
    @constraint(model, [i=1:P,t=1:T,k=1:K], j[i,t,k] <= s[i,t,k])
    @constraint(model, [i=1:P,t=1:T,k=1:K], j[i,t,k] <= D[i,t])
    # cannot spend more than we have in capital at beginning of week
    #@constraint(model, [t=1:T,k=1:K], lambda[t,k] <= m[t,k])   # capital no longer constrained
    # inventory in week is equal to prev week inv - what we sold last week + what we ordered two weeks ago
    @constraint(model, [i=1:P,t=leadtime:T-1,k=1:K], s[i,t+1,k] == s[i,t,k] - j[i,t,k] + x[i,t-1])
    @constraint(model, [i=1:P,k=1:K], s[i,2,k] == s[i,1,k] - j[i,1,k])
    @constraint(model, [i=1:P,k=1:K], s[i,1,k] == startingInv[i])
    # starting Inv (T=14) must be enough to start next quarter (whatever is left after last week
    # + what we ordered in second to last week)
    @constraint(model, [i=1:P,k=1:K], s[i,T,k] - j[i,T,k] + x[i,T-1] >= nextQstartingInv[i])
    
    @objective(model, Max, sum(neighborWeight[k]*(m[T,k] + sum(j[i,t,k]*(r[i] - f[i]) for i=1:P,t=T-1) - lambda[T,k]) for k=1:K))

    optimize!(model)

    #termination_status(model)
    return objective_value(model), value.(x)
    
end

runModelPres (generic function with 1 method)

In [195]:
function findKNN(cass_data_in, test_point_in, k)
   
    cass_data = deepcopy(cass_data_in)
    test_point = deepcopy(test_point_in)
    
    cass_mean = mean(cass_data[!, :avgCASS]) 
    cass_std = std(cass_data[!, :avgCASS])
     
    #select!(cass_data, Not(:Column1))
    cass_data = append!(cass_data, test_point)

    # normalize columns (except for year and quarter)
    for col in names(cass_data)
        if col == "Year" || col == "Quarter"
            continue
        end
        cass_data[!, col] = (cass_data[!, col] .- mean(cass_data[!, col])) ./ std(cass_data[!, col])
        #test_point[!, col] = (test_point[!, col] .- mean(test_point[!, col])) ./ std(test_point[!, col])
    end
    
    test_point = DataFrame(cass_data[size(cass_data)[1], :])
    X = Array(select(cass_data[1:(size(cass_data)[1]-1), :], Not(:avgCASS)))'
    kd_tree = KDTree(X)
    test_point = Array(select(test_point, Not(:avgCASS)))'
    
    id, dist = knn(kd_tree, test_point, k)
    knn_predictions = cass_data[id[1],:].avgCASS
    knn_predictions = knn_predictions .* cass_std .+ cass_mean
    return knn_predictions
end

findKNN (generic function with 1 method)

In [196]:
# HOW TO CHECK PRESCRIPTION AGAINST REALITY
# should theoretically be able to set x = x_planned in the opt model
# would be true U

function checkPlan(startingCap, startingInv, nextQstartingInv, D, U, planned_x)

    P = size(D)[1]
    T = size(D)[2]

    modelReality = Model(with_optimizer(Gurobi.Optimizer,TimeLimit=60, gurobi_env));
    set_optimizer_attribute(modelReality, "OutputFlag", 0)

    @variable(modelReality, x[i=1:P, t=1:T] >= 0) # main decision
    @variable(modelReality, j[i=1:P, t=1:T] >= 0) 
    @variable(modelReality, s[i=1:P, t=1:T] >= 0) 
    @variable(modelReality, m[t=1:T])
    @variable(modelReality, lambda[t=1:T] >= 0)

    # set x = planned x
    @constraint(modelReality, [i=1:P,t=1:T], x[i,t] == planned_x[i,t])
    
    # profit (capital) in every week = last week's + 2 weeks ago's sales - this weeks cost
    @constraint(modelReality, [t=1:T-2], m[t+2] == m[t+1] + sum(j[i,t]*(r[i] - f[i]) for i=1:P) - lambda[t+1])
    @constraint(modelReality, m[1] == startingCap)
    @constraint(modelReality, m[2] == m[1] - lambda[1])
    # cost in week is equal to purchasing cost + trans cost + storage cost
    @constraint(modelReality, [t=1:T], lambda[t] == sum(x[i,t]*c[i] for i=1:P) + U*sum(x[i,t]*w[i] for i=1:P) + sum(s[i,t]*FBA[i] for i=1:P))
    # sales capped by storage and demand
    @constraint(modelReality, [i=1:P,t=1:T], j[i,t] <= s[i,t])
    @constraint(modelReality, [i=1:P,t=1:T], j[i,t] <= D[i,t])
    # cannot spend more than we have in capital at beginning of week
    #@constraint(modelNoPres, [t=1:T], lambda[t] <= m[t])
    # inventory in week is equal to prev week inv - what we sold last week + what we ordered two weeks ago
    @constraint(modelReality, [i=1:P,t=leadtime:T-1], s[i,t+1] == s[i,t] - j[i,t] + x[i,t-1])
    @constraint(modelReality, [i=1:P], s[i,2] == s[i,1] - j[i,1])
    @constraint(modelReality, [i=1:P], s[i,1] == startingInv[i])
    # ending inventory (T=14) must be enough to start next quarter (whatever is left after last week
    # + what we ordered in the second to last week)
    @constraint(modelReality, [i=1:P], s[i,T] - j[i,T] + x[i,T-1] >= nextQstartingInv[i])
    
    @objective(modelReality, Max, m[T] + sum(j[i,T-1]*(r[i] - f[i]) for i=1:P) - lambda[T])
    #@objective(modelNoPres, Max, m[T])
    
    optimize!(modelReality)

    #termination_status(model)
    return objective_value(modelReality)
    
end

checkPlan (generic function with 1 method)

## Q1

In [214]:
# starting capital and inventory are 0
startingCap1 = 0
Q1_startingInv = zeros(19)

# PRESCRIPTION
U1_pres = findKNN(trainQ1, testQ1, 5)
Q1Prof_pres_est, Q1_pres_x = runModelPres(startingCap1, Q1_startingInv, Q2_startingInv, d1, U1_pres);

# ORACLE
U1_true = testQ1[1,7]
Q1Prof_oracle_est, Q1_oracle_x = runModelPres(startingCap1, Q1_startingInv, Q2_startingInv, d1, U1_true);

# LAST Q
U1_lastQ = trainQ1[end,7]
Q1Prof_lastQ_est, Q1_lastQ_x = runModelPres(startingCap1, Q1_startingInv, Q2_startingInv, d1, U1_lastQ);


println("Estimated Profits: ")
println(" - Prescription: \$", Q1Prof_pres_est)
println(" - Oracle:       \$", Q1Prof_oracle_est)
println(" - Last Quarter: \$", Q1Prof_lastQ_est)

Estimated Profits: 
 - Prescription: $1.4886854495214005e6
 - Oracle:       $1.5056136948115781e6
 - Last Quarter: $1.500018490948422e6


### Q1. Check how plans perform in reality

In [215]:
Q1Prof_pres_real = checkPlan(startingCap1, Q1_startingInv, Q2_startingInv, d1, U1_true, Q1_pres_x)
Q1Prof_oracle_real = checkPlan(startingCap1, Q1_startingInv, Q2_startingInv, d1, U1_true, Q1_oracle_x)
Q1Prof_lastQ_real = checkPlan(startingCap1, Q1_startingInv, Q2_startingInv, d1, U1_true, Q1_lastQ_x)

println("Realized Profits: ")
println(" - Prescription: \$", Q1Prof_pres_real)
println(" - Oracle:       \$", Q1Prof_oracle_real)
println(" - Last Quarter: \$", Q1Prof_lastQ_real)

Realized Profits: 
 - Prescription: $1.505613694811578e6
 - Oracle:       $1.505613694811578e6
 - Last Quarter: $1.505613694811578e6


## Q2

In [118]:
# PRESCRIPTION
startingCap2_pres = Q1Prof_pres_real
U2_pres = findKNN(trainQ2, testQ2, 5)
Q2Prof_pres_est, Q2_pres_x = runModelPres(startingCap2_pres, Q2_startingInv, Q3_startingInv, d2, U2_pres);

# ORACLE
startingCap2_oracle = Q1Prof_oracle_real
U2_true = testQ2[1,7]
Q2Prof_oracle_est, Q2_oracle_x = runModel(startingCap2_oracle, Q2_startingInv, Q3_startingInv, d2, U2_true);

# LAST Q
startingCap2_lastQ = Q1Prof_lastQ_real
U2_lastQ = trainQ2[end,7]
Q2Prof_lastQ_est, Q2_lastQ_x = runModel(startingCap2_lastQ, Q2_startingInv, Q3_startingInv, d2, U2_lastQ);


println("Estimated Profits: ")
println(" - Prescription: \$", Q2Prof_pres_est)
println(" - Oracle:       \$", Q2Prof_oracle_est)
println(" - Last Quarter: \$", Q2Prof_lastQ_est)

Estimated Profits: 
 - Prescription: $5.384191656107818e6
 - Oracle:       $5.430483524186049e6
 - Last Quarter: $5.408877475546335e6


### Q2. Check how plans perform in reality

In [119]:
Q2Prof_pres_real = checkPlan(startingCap2_pres, Q2_startingInv, Q3_startingInv, d2, U2_true, Q2_pres_x)
Q2Prof_oracle_real = checkPlan(startingCap2_oracle, Q2_startingInv, Q3_startingInv, d2, U2_true, Q2_oracle_x)
Q2Prof_lastQ_real = checkPlan(startingCap2_lastQ, Q2_startingInv, Q3_startingInv, d2, U2_true, Q2_lastQ_x)

println("Realized Profits: ")
println(" - Prescription: \$", Q2Prof_pres_real)
println(" - Oracle:       \$", Q2Prof_oracle_real)
println(" - Last Quarter: \$", Q2Prof_lastQ_real)

Realized Profits: 
 - Prescription: $5.419515856030204e6
 - Oracle:       $5.43048352418605e6
 - Last Quarter: $5.427998424199156e6


## Q3

In [120]:
# PRESCRIPTION
startingCap3_pres = Q2Prof_pres_real
U3_pres = findKNN(trainQ3, testQ3, 5)
Q3Prof_pres_est, Q3_pres_x = runModelPres(startingCap3_pres, Q3_startingInv, Q4_startingInv, d3, U3_pres);

# ORACLE
startingCap3_oracle = Q2Prof_oracle_real
U3_true = testQ3[1,7]
Q3Prof_oracle_est, Q3_oracle_x = runModel(startingCap3_oracle, Q3_startingInv, Q4_startingInv, d3, U3_true);

# LAST Q
startingCap3_lastQ = Q2Prof_lastQ_real
U3_lastQ = trainQ3[end,7]
Q3Prof_lastQ_est, Q3_lastQ_x = runModel(startingCap3_lastQ, Q3_startingInv, Q4_startingInv, d3, U3_lastQ);


println("Estimated Profits: ")
println(" - Prescription: \$", Q3Prof_pres_est)
println(" - Oracle:       \$", Q3Prof_oracle_est)
println(" - Last Quarter: \$", Q3Prof_lastQ_est)

Estimated Profits: 
 - Prescription: $8.0190412060928475e6
 - Oracle:       $8.00972153324605e6
 - Last Quarter: $8.027429456879156e6


### Q3. Check how plans perform in reality

In [124]:
#Q3Prof_pres_real = checkPlan(startingCap3_pres, Q3_startingInv, Q4_startingInv, d3, U3_true, Q3_pres_x)
Q3Prof_oracle_real = checkPlan(startingCap3_oracle, Q3_startingInv, Q4_startingInv, d3, U3_true, Q3_oracle_x)
#Q3Prof_lastQ_real = checkPlan(startingCap3_lastQ, Q3_startingInv, Q4_startingInv, d3, U3_true, Q3_lastQ_x)

println("Realized Profits: ")
#println(" - Prescription: \$", Q3Prof_pres_real)
println(" - Oracle:       \$", Q3Prof_oracle_real)
#println(" - Last Quarter: \$", Q3Prof_lastQ_real)

Realized Profits: 
 - Oracle:       $8.00972153324605e6


In [92]:
# get quarter specific data
startingCap3 = endQ2profit_real

U3 = findKNN(trainQ3, testQ3, 5)

# run model for q1
endQ3profit, Q3_x = runModelPres(startingCap3, Q3_startingInv, Q4_startingInv, d3, U3);

Academic license - for non-commercial use only - expires 2022-08-19


In [93]:
endQ3profit_est

8.0190412060928475e6

In [94]:
# test against true U
startingCap3_oracle = endQ2profit_oracle_real
U3_true = testQ3[1,7]

endQ3profit_oracle, Q3_x_oracle = runModel(startingCap3_oracle, Q3_startingInv, Q4_startingInv, d3, U3_true);

Academic license - for non-commercial use only - expires 2022-08-19


In [95]:
endQ3profit_oracle

8.00972153324605e6

In [83]:
# test against prediction of U - just last quarter's U
startingCap3_lastQ = endQ2profit_lastQ_real

U3_lastQ = trainQ3[end,7]

endQ3profit_lastQ, Q3_x_lastQ = runModel(startingCap3_lastQ, Q2_startingInv, Q3_startingInv, d2, U2_lastQ);

Academic license - for non-commercial use only - expires 2022-08-19


In [84]:
endQ2profit_lastQ

5.408877475546335e6

### Q3. Check how plans perform in reality

In [85]:
endQ2profit_real = checkPlan(startingCap2, Q2_startingInv, Q3_startingInv, d2, U2_true, Q2_x)

Academic license - for non-commercial use only - expires 2022-08-19


5.419515856030204e6

In [86]:
endQ2profit_oracle_real = checkPlan(startingCap2_oracle, Q2_startingInv, Q3_startingInv, d2, U2_true, Q2_x_oracle)

Academic license - for non-commercial use only - expires 2022-08-19


5.43048352418605e6

In [87]:
endQ2profit_lastQ_real = checkPlan(startingCap2_lastQ, Q2_startingInv, Q3_startingInv, d2, U2_true, Q2_x_lastQ)

Academic license - for non-commercial use only - expires 2022-08-19


5.427998424199156e6

In [255]:
## WITH PRESCRIPTION

# model function
function runModelPres(startingCap, startingInv, nextQstartingInv, e, D, U)

    model = Model(with_optimizer(Gurobi.Optimizer,TimeLimit=60, gurobi_env));
    set_optimizer_attribute(model, "OutputFlag", 0)
    
    if isa(U, Vector) == true
        K = size(U)[1]
    else
        K = 1
    end
    
    if isa(e, Vector) == true
        L = 1
    else
        L = size(e)[2]
    end
    
    kweight = zeros(K)
    kweight .= 1/K
    
    lweight = zeros(L)
    lweight .= 1/L
    
    P = size(D)[1]
    T = size(D)[2]

    @variable(model, x[i=1:P, t=1:T] >= 0) # Main decision
    @variable(model, j[i=1:P, t=1:T, k=1:K, l=1:L] >= 0) 
    @variable(model, s[i=1:P, t=1:T, k=1:K, l=1:L] >= 0) 
    @variable(model, m[t=1:T, k=1:K, l=1:L])
    @variable(model, lambda[t=1:T, k=1:K, l=1:L] >= 0)
    @variable(model, notDemand[i=1:P,t=1:T,k=1:K, l=1:L] >=0)

    # profit (capital) in every week = last week's + 2 weeks ago's sales - this weeks cost
    @constraint(model, [t=1:T-2, k=1:K, l=1:L], m[t+2,k,l] == m[t+1,k,l] + sum(j[i,t,k,l]*(r[i] - f[i]) for i=1:P) - lambda[t+1,k,l])
    @constraint(model, [k=1:K, l=1:L], m[1,k,l] == startingCap)
    @constraint(model, [k=1:K, l=1:L], m[2,k,l] == m[1,k,l] - lambda[1,k,l])
    # cost in week is equal to purchasing cost + trans cost + storage cost
    @constraint(model, [t=1:T,k=1:K, l=1:L], lambda[t,k,l] == sum(x[i,t]*c[i] for i=1:P) + U[k]*sum(x[i,t]*w[i] for i=1:P) + sum(s[i,t,k,l]*FBA[i] for i=1:P) + sum(notDemand[i,t,k,l] for i=1:P))
    # sales capped by storage and demand
    @constraint(model, [i=1:P,t=1:T,k=1:K, l=1:L], j[i,t,k,l] <= s[i,t,k,l])
    @constraint(model, [i=1:P,t=1:T,k=1:K, l=1:L], j[i,t,k,l] <= e[i,l]*D[i,t])
    # cannot spend more than we have in capital at beginning of week
    #@constraint(model, [t=1:T,k=1:K], lambda[t,k] <= m[t,k])   # capital no longer constrained
    # inventory in week is equal to prev week inv - what we sold last week + what we ordered two weeks ago
    @constraint(model, [i=1:P,t=leadtime:T-1,k=1:K, l=1:L], s[i,t+1,k,l] == s[i,t,k,l] - j[i,t,k,l] + x[i,t-1])
    @constraint(model, [i=1:P,k=1:K, l=1:L], s[i,2,k,l] == s[i,1,k,l] - j[i,1,k,l])
    @constraint(model, [i=1:P,k=1:K, l=1:L], s[i,1,k,l] == startingInv[i])
    # starting Inv (T=14) must be enough to start next quarter (whatever is left after last week
    # + what we ordered in second to last week)
    @constraint(model, [i=1:P,k=1:K, l=1:L], s[i,T,k,l] - j[i,T,k,l] + x[i,T-1] >= nextQstartingInv[i])
    @constraint(model, [i=1:P,t=1:T,k=1:K, l=1:L], notDemand[i,t,k,l] >= e[i,l]*D[i,t] - j[i,t,k,l])
    
    @objective(model, Max, sum(lweight[l]*sum(kweight[k]*(m[T,k,l] + sum(j[i,t,k,l]*(r[i] - f[i]) for i=1:P,t=T-1) - lambda[T,k,l]) for k=1:K) for l=1:L))

    optimize!(model)

    #termination_status(model)
    return objective_value(model), value.(x), value.(notDemand)
    
end

runModelPres (generic function with 2 methods)

In [279]:
e1_pres = Matrix(CSV.read("data/test_e_matrix.csv", DataFrame, header=false))
e1_true = Matrix(CSV.read("data/test_e_true.csv", DataFrame, header=false))
e1_lastQ = Vector(CSV.read("data/test_e_rand.csv", DataFrame, header=false)[:,1])

19-element Vector{Float64}:
 0.655734541
 0.86982255
 1.2814704
 1.249817154
 1.463384874
 1.440950714
 1.118047349
 0.631509619
 1.229456726
 0.934489525
 0.704587291
 1.240693616
 1.372970192
 0.955798501
 1.479389604
 0.929065942
 0.871394064
 1.151336744
 0.961378016

In [280]:
# starting capital and inventory are 0
startingCap1 = 0
Q1_startingInv = zeros(19)

# PRESCRIPTION
U1_pres = findKNN(trainQ1, testQ1, 5)
Q1Prof_pres_est, Q1_pres_x, Q1_pres_notD = runModelPres(startingCap1, Q1_startingInv, Q2_startingInv, e1_pres, d1, U1_pres);

# ORACLE
U1_true = testQ1[1,7]
Q1Prof_oracle_est, Q1_oracle_x, Q1_pres_notD = runModelPres(startingCap1, Q1_startingInv, Q2_startingInv, e1_true, d1, U1_true);

# LAST Q
U1_lastQ = trainQ1[end,7]
Q1Prof_lastQ_est, Q1_lastQ_x, Q1_pres_notD = runModelPres(startingCap1, Q1_startingInv, Q2_startingInv, e1_lastQ, d1, U1_lastQ);


println("Estimated Profits: ")
println(" - Prescription: \$", Q1Prof_pres_est)
println(" - Oracle:       \$", Q1Prof_oracle_est)
println(" - Last Quarter: \$", Q1Prof_lastQ_est)

Estimated Profits: 
 - Prescription: $846618.438547624
 - Oracle:       $1.4981984096115427e6
 - Last Quarter: $1.1572314482571208e6


In [285]:
d1

19×13 Matrix{Any}:
  34   49   40   65   50   58   61   58   70   86   67   75   66
  57   78   68   61   78  118  109   99   97  109  111  108  107
  75   88   97   97   91  117  116  140  120  161  138  127  126
  57   75   73   69   65  102  117  117   86   97  123  125  133
 211  304  335  360  327  407  437  426  487  498  516  522  471
 253  358  364  376  398  427  557  525  457  527  564  577  588
 128  188  184  166  217  246  254  262  260  286  331  289  303
 100  159  161  148  158  212  215  196  225  279  262  289  271
  39   46   60   58   57   68   90   76   88   81   84   75   86
  55   68   71   72   84   94   92   94  104  108   96  102  108
  58   80   73  105  104  131   96  102  111  116  116  114  134
   8    9   11    8    5   11   11    6    9   11    9   12   14
   8    8    8   10   12   12    7    9   12   15   11   17   14
 204  264  242  264  295  337  395  376  402  406  447  434  393
  37   49   49   57   58   63   73   72   75   92   69   85   97
  40  

In [283]:
Q1_lastQ_x

19×13 Matrix{Float64}:
  38.4551    62.4896    48.0689    55.7599    58.6441    55.7599   …   82.6785   64.4123    72.1034  0.0   185.0  0.0
  65.3737    58.6441    74.9875   113.443    104.79      95.1764      121.867   124.103    120.749   0.0   249.0  0.0
  92.7125    92.7125    86.9777   111.828    111.52     134.593       154.782   132.67     122.095   0.0   330.0  0.0
  70.1806    66.3351    62.4896    98.0606   112.481    112.481        93.2537  118.249    120.172   0.0   318.0  0.0
 429.293    461.329    419.041    521.558    560.003    584.885       683.739   708.453    716.69    0.0  1316.0  0.0
 516.971    541.797    573.498    615.286    802.61     756.499    …  759.381   812.696    831.429   0.0  1307.0  0.0
 205.721    185.596    242.616    275.04     283.984    292.928       319.762   370.074    323.116   0.0   702.0  0.0
 180.006    165.471    176.651    237.026    240.38     219.137       311.935   292.928    323.116   0.0   607.0  0.0
  57.3479    55.7599    54.7985  

In [259]:
Q1_pres_notD[:,:,1,1]

19×13 Matrix{Float64}:
  30.6   44.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   59.4
  51.3   70.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   96.3
  67.5   79.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  113.4
  51.3   67.5  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  119.7
 189.9  273.6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0
 227.7  322.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0    0.0
 115.2  169.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  272.7
  90.0  143.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  243.9
  35.1   41.4  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   77.4
  49.5   61.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   97.2
  52.2   72.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  120.6
   7.2    8.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   12.6
   7.2    7.2  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   12.6
 183.6  237.6  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

In [281]:
Q1Prof_pres_real = checkPlan(startingCap1, Q1_startingInv, Q2_startingInv, d1, U1_true, Q1_pres_x)
Q1Prof_oracle_real = checkPlan(startingCap1, Q1_startingInv, Q2_startingInv, d1, U1_true, Q1_oracle_x)
Q1Prof_lastQ_real = checkPlan(startingCap1, Q1_startingInv, Q2_startingInv, d1, U1_true, Q1_lastQ_x)

println("Realized Profits: ")
println(" - Prescription: \$", Q1Prof_pres_real)
println(" - Oracle:       \$", Q1Prof_oracle_real)
println(" - Last Quarter: \$", Q1Prof_lastQ_real)

Realized Profits: 
 - Prescription: $864477.4488853998
 - Oracle:       $1.5051164096115427e6
 - Last Quarter: $1.2796134425922069e6
